In [1]:
DEFAULT_FILENAME = "input19.txt"

workflow = {'A': lambda _ : True, 'R': lambda _ : False}

def convert_to_lambda_str(ins):
    if len(ins) == 1:
        return f"(lambda part: workflow['{ins[0]}'](part))"
    condition, key = ins[0].split(":")
    tail = ins[1:]
    condition = f"(lambda part: workflow['{key}'](part) if part['{condition[0]}']{condition[1:]} else {convert_to_lambda_str(tail)}(part))"
    return condition

def load_file(filename = DEFAULT_FILENAME):
    global workflow
    parts = []
    with open(filename, "r") as f:
        workflow_loaded = False
        for li in f:
            if len(li.strip()) == 0:
                workflow_loaded = True
                continue
            if workflow_loaded:
                li = li.replace("=", ":")
                for c in "xmas":
                    li = li.replace(c, f"'{c}'")
                parts.append(eval(li))
            else:
                key = li.split("{")[0]
                ins = li.split("{")[1].strip()[:-1].split(',')
                workflow[key] = eval(convert_to_lambda_str(ins))
    return parts

parts = load_file()

accepted_sum = 0
for part in parts:
    if workflow["in"](part):
        for c in "xmas":
            accepted_sum += part[c]

# life is a lambda
print(f"Part 1: {accepted_sum}")

Part 1: 446935


In [2]:
class day_19_part_2:
  def load_file(self, filename = DEFAULT_FILENAME):
      self.parts = []
      self.workflow = {}
      with open(filename, "r") as f:
          for li in f:
              if len(li.strip()) == 0:
                  break
              key = li.split("{")[0]
              ins = li.split("{")[1].strip()[:-1].split(',')
              self.workflow[key] = ins
  @staticmethod
  def ranges_comb(ranges):
    total_comb = 0
    for r in ranges:
      this_comb = 1
      for start, end in r:
        this_comb *= (end - start + 1)
      total_comb += this_comb
    return total_comb
  @staticmethod
  def ranges_overlap(r1, r2):
    if r1[0] > r2[1] or r2[0] > r1[1]:
      return (0, -1)
    return (max(r1[0], r2[0]), min(r1[1], r2[1]))
  def handle_workflow(self, key, ranges):
    if key == 'A':
      return ranges
    elif key == 'R':
      return []
    return self.handle_instructions(self.workflow[key], ranges)
  def handle_instructions(self, ins, ranges):
    if len(ins) == 1:
      return self.handle_workflow(ins[0], ranges)
    condition, key = ins[0].split(":")
    tail = ins[1:]
    new_ranges = []
    for i, c in enumerate("xmas"):
      if c in condition:
        thres = int(condition[2:])
        true_r, false_r = ((thres + 1, 4000), (0, thres)) if ">" in condition else ((0, thres - 1), (thres, 4000))
        true_ranges = [[self.ranges_overlap(true_r, ranges_iter[ii]) if i == ii else ranges_iter[ii] for ii in range(4)] for ranges_iter in ranges]
        false_ranges = [[self.ranges_overlap(false_r, ranges_iter[ii]) if i == ii else ranges_iter[ii] for ii in range(4)] for ranges_iter in ranges]
        new_ranges += self.handle_workflow(key, true_ranges)
        new_ranges += self.handle_instructions(tail, false_ranges)
        break
    return [r for r in new_ranges if day_19_part_2.ranges_comb([r])]

sol2 = day_19_part_2()
sol2.load_file()
ranges = sol2.handle_workflow("in", [[(1, 4000), (1, 4000), (1, 4000), (1, 4000)]])
total_comb = day_19_part_2.ranges_comb(ranges)
# still recursive
print(f"Part 2: {total_comb}")

Part 2: 141882534122898
